In [1]:
%load_ext autoreload
%autoreload 2    

# Setup

In [27]:
DB_NAME = "./memories.db"
# DB_NAME = "./test.db"

In [28]:
from broai.prompt_management.core import PromptGenerator
from broai.prompt_management.interface import Persona, Instructions, Examples, Example
from pydantic import BaseModel, Field
from typing import List
from broai.experiments.bro_agent import BroAgent
import json
from broai.interface import Context, Contexts
from broai.experiments.vector_store import DuckVectorStore

In [29]:
from package.jargon_store import JargonStore, JargonRecord

In [30]:
from broai.experiments.cross_encoder import ReRanker
rr = ReRanker()

/tmp/ipykernel_437/2186289470.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [31]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/tmp/ipykernel_437/2466456318.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 171663.19it/s]


In [32]:
raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)
jargon_memory = JargonStore(db_name=DB_NAME, table="jargon_memory")

/tmp/ipykernel_437/1440006260.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_437/1440006260.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
/tmp/ipykernel_437/1440006260.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)


# Agent Flows: 
- JargonDetector
- JargonEditor

In [33]:
from package.online_flow import OnlineFlow, JargonFlow, KnowledgeFlow, Payload
from agents.jargon_detector import JargonDetector
from agents.jargon_editor import JargonEditor
from agents.query_decomposer import QueryDecomposer
from agents.ai_oracle import AIOracle
from agents.kb_oracle import KBOracle

In [39]:
jargon_flow = JargonFlow(jargon_memory=jargon_memory, jargon_detector=JargonDetector(), jargon_editor=JargonEditor())
knowledge_flow = KnowledgeFlow(longterm_memory=longterm_memory, reranker=rr, search_method="vector")
olf = OnlineFlow(
    oracle=KBOracle(), 
    knowledge_flow=knowledge_flow,
    jargon_flow=jargon_flow,
    query_decomposer=QueryDecomposer(),
)

In [40]:
%%time

# olf.run(message="What is the dataset used in the study?")
# olf.run(message="What does STORM do in the study?")
# olf.run(message="What does STORM stand for?")
# olf.run(message="What does STORM stand for and how does it work? Explain it in step-by-step manner.")
olf.run(message="What does STORM stand for and how does it work? Could you please explain it like I'm a five years old? I'm really new to this thing.")

print(olf.payload.answer)

Let me explain STORM in a super simple way.

Imagine you want to write a big article about a topic, like dinosaurs. But, you don't know where to start. That's where STORM comes in. STORM is like a special tool that helps you write a great article by doing some of the hard work for you.

STORM stands for Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking. It's a big phrase, but don't worry, I'll break it down for you.

"Synthesis" means combining different things together to make something new. In this case, STORM combines different ideas and information to help you write a great article.

"Topic Outlines" means creating a plan or a map of what you want to write about. STORM helps you create a good plan by asking you questions and finding information about your topic.

"Retrieval" means finding information from different places, like books or the internet. STORM uses this information to help you write a great article.

"Multi-perspective Question Asking"

In [38]:
[context.metadata['sequence'] for context in olf.payload.reranked_contexts]

[3, 1, 2, 9, 22, 8, 27, 39, 46, 20]

In [11]:
[context.metadata['sequence'] for context in olf.payload.reranked_contexts]

[3, 1, 9, 8, 27, 39, 46, 20, 11, 10]

In [25]:
[context.metadata['sequence'] for context in olf.payload.reranked_contexts]

[1, 3, 20, 22, 9, 27, 8, 10, 46, 39]